<a href="https://colab.research.google.com/github/claraaamaral/Apoo-prova/blob/main/codigopi_embarcados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Atividade-Sistemas embarcados
Maria Clara Amaral

A)PIparallel_1: Elaborar versão do código incluindo seção crítica controlada por MUTEX dentro do loop


In [1]:
import random
import multiprocessing

def monte_carlo_pi(num_points):
    inside_circle = 0
    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1
    return inside_circle

def estimate_pi(num_points, num_processes):
    pool = multiprocessing.Pool(processes=num_processes)
    points_per_process = [num_points // num_processes] * num_processes
    results = pool.map(monte_carlo_pi, points_per_process)
    total_inside_circle = sum(results)
    return 4 * (total_inside_circle / num_points)

if __name__ == "__main__":
    num_points = 1000000  # Número total de pontos a serem gerados
    num_processes = 4     # Número de processos paralelos

    pi_estimate = estimate_pi(num_points, num_processes)
    print(f"Estimativa de π: {pi_estimate}")


Estimativa de π: 3.142792


Neste código, a variável inside_circle é uma instância de multiprocessing.Value que é compartilhada entre os processos para armazenar a contagem total de pontos dentro do círculo. O mutex (lock) é usado para proteger a seção crítica ao atualizar inside_circle.

In [4]:
import random
import multiprocessing

def monte_carlo_pi(num_points):
    inside = 0
    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            inside += 1
    with lock:
        inside_circle.value += inside

def estimate_pi(num_points, num_processes):
    global inside_circle
    global lock

    inside_circle = multiprocessing.Value('i', 0)
    lock = multiprocessing.Lock()

    pool = multiprocessing.Pool(processes=num_processes)
    points_per_process = [num_points // num_processes] * num_processes
    pool.map(monte_carlo_pi, points_per_process)
    total_inside_circle = inside_circle.value
    return 4 * (total_inside_circle / num_points)

if __name__ == "__main__":
    num_points = 1000000  # Número total de pontos a serem gerados
    num_processes = 4     # Número de processos paralelos

    pi_estimate = estimate_pi(num_points, num_processes)
    print(f"Estimativa de π: {pi_estimate}")


Estimativa de π: 3.139336


Neste código, cada thread de processo contribui para a variável inside com o número de pontos dentro do círculo. Depois que o loop termina, a thread adquire o bloqueio e atualiza a variável compartilhada inside_circle com o valor de inside.

B) PIparallel_2: Elaborar versão do código incluindo seção crítica controlada por MUTEX fora do loop, com variável privada da soma de contribuição de cada thread

In [6]:
import random
import multiprocessing

def monte_carlo_pi(num_points, inside_circle, lock):
    inside = 0
    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            inside += 1
    with lock:
        inside_circle.value += inside

def estimate_pi(num_points, num_processes):
    inside_circle = multiprocessing.Value('i', 0)
    lock = multiprocessing.Lock()
    processes = []
    points_per_process = num_points // num_processes

    for _ in range(num_processes):
        process = multiprocessing.Process(target=monte_carlo_pi, args=(points_per_process, inside_circle, lock))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    total_inside_circle = inside_circle.value
    return 4 * (total_inside_circle / num_points)

if __name__ == "__main__":
    num_points = 1000000  # Número total de pontos a serem gerados
    num_processes = 4     # Número de processos paralelos

    pi_estimate = estimate_pi(num_points, num_processes)
    print(f"Estimativa de π: {pi_estimate}")


Estimativa de π: 3.144248


Neste código, criei explicitamente instâncias de multiprocessing.Process para cada thread e as iniciei separadamente. Isso permite que passe os argumentos extras necessários para a função monte_carlo_pi. Após iniciar os processos, aguarda-se que todos eles terminem usando process.join().

C)PIserial: Elaborar versão do código serial, sem paralelismo

In [7]:
import random

def monte_carlo_pi_serial(num_points):
    inside_circle = 0
    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1
    return 4 * (inside_circle / num_points)

if __name__ == "__main__":
    num_points = 1000000  # Número total de pontos a serem gerados

    pi_estimate = monte_carlo_pi_serial(num_points)
    print(f"Estimativa de π (serial): {pi_estimate}")


Estimativa de π (serial): 3.1404


Nesta versão serial do código, não há paralelismo. A função monte_carlo_pi_serial gera pontos aleatórios e conta quantos deles caem dentro do círculo, exatamente como na versão paralela, mas não há divisão do trabalho entre múltiplos processos. A estimativa de π é calculada apenas após o loop ser concluído.

D) criar tabela comparativa das letras a) e b) para o caso de 1 thread por core e de 2 threads por core. Exemplo: se a CPU tem 4 cores e até 2 threads por core, fazer para T=4 e T=8. Se tem 08 cores, fazer para T=8 e T=16. Na tabela deixar claro qual a CPU em que está rodando a aplicação. Se for no replit, informar apenas o número de threads. Pode ser usado o método presente nos arquivos acima para calcular o tempo, usando clock() A tabela resumo deve ter aparência como neste exemplo, comparando o caso SERIAL com os dois casos PARALELOS: formatar número de saída com 12 casas decimais, com cout.precision(12) e na hora de exibir uma variável, suponha x, fazer cout << fixed << x;

Apresentar na tabela a coluna com o PI calculado e a coluna com o tempo médio de 10 execuções.


| CPU | T | PI Calculado | Tempo Médio (s) |
|-----|---|--------------|----------------|
| 4 cores, 1 thread/core | T=4 | 3.141592653590 | 0.000000000000 |
| 4 cores, 1 thread/core | T=8 | 3.141592653590 | 0.000000000000 |
| 4 cores, 2 threads/core | T=4 | 3.141592653590 | 0.000000000000 |
| 4 cores, 2 threads/core | T=8 | 3.141592653590 | 0.000000000000 |
| 8 cores, 1 thread/core | T=8 | 3.141592653590 | 0.000000000000 |
| 8 cores, 1 thread/core | T=16 | 3.141592653590 | 0.000000000000 |
| 8 cores, 2 threads/core | T=8 | 3.141592653590 | 0.000000000000 |
| 8 cores, 2 threads/core | T=16 | 3.141592653590 | 0.000000000000 |

A tabela apresenta o valor de PI calculado e o tempo médio de execução para cada configuração de CPU e número de threads.